# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.u

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355078177024                   -1.10    6.2    156ms
  2   -9.356774933691       -2.77       -1.43    1.0   88.5ms
  3   -9.357077851182       -3.52       -2.77    2.2   97.1ms
  4   -9.357119110113       -4.38       -2.98    6.1    524ms
  5   -9.357119646743       -6.27       -3.12    1.1    102ms
  6   -9.357119987637       -6.47       -3.27    4.1    119ms
  7   -9.357120195491       -6.68       -3.45    1.0   80.2ms
  8   -9.357120270954       -7.12       -3.61    1.0   79.5ms
  9   -9.357120312074       -7.39       -3.89    1.1   86.5ms
 10   -9.357120321126       -8.04       -4.16    1.1   79.4ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.74297611403                   -0.97    6.6    523ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     1
│     1
│     1
│     6
│     1
│     1
│     1
│     1
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  2   -18.73842412202   +   -2.34       -1.37    1.6    278ms
  3   -18.79038276637       -1.28       -1.75    4.0    325ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      1
│     13
│      1
│      1
│      1
│      3
│      1
│      4
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  4   -18.79256505253       -2.66       -1.95    3.1    331ms
  5   -18.79260527377       -4.40       -2.69    2.6    296ms
  6   -18.79262155550       -4.79       -3.05    7.9    535ms
  7   -18.79262316120       -5.79       -3.18    1.9    271ms
  8   -18.7

In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.52547244876                   -0.84    7.9    1.76s
  2   -37.55592181876       -1.52       -1.21    2.8    795ms
  3   -37.56457829625       -2.06       -2.09   12.8    1.89s
  4   -37.56487185172       -3.53       -2.36    9.2    1.50s
  5   -37.56497788739       -3.97       -3.36    1.9    904ms
  6   -37.56498347955       -5.25       -3.59    9.9    1.92s
  7   -37.56498511302       -5.79       -4.09    2.1    1.16s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355333963310                   -1.10    6.2    200ms
  2   -9.356861106271       -2.82       -1.90    1.0   68.2ms
  3   -9.357105453673       -3.61       -2.84    3.1    122ms
  4   -9.357106369707       -6.04       -2.69    5.9    172ms
  5   -9.357119629722       -4.88       -3.33    1.1   68.4ms
  6   -9.357120249096       -6.21       -3.89    1.5   76.5ms
  7   -9.357120320244       -7.15       -4.53    4.4    131ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.53942186086                   -0.84   10.5    2.07s
  2   -37.52844995690   +   -1.96       -1.45    2.5    680ms
  3   -16.99438811456   +    1.31       -0.40    9.2    1.80s
  4   -33.62464754581        1.22       -0.76    8.5    1.79s
  5   -37.41565446775        0.58       -1.31    6.1    1.66s
  6   -33.86564159810   +    0.55       -0.78    6.0    1.44s
  7   -37.52851170466        0.56       -1.73    6.0    1.45s
  8   -37.55424612721       -1.59       -1.97    1.4    753ms
  9   -37.56199729930       -2.11       -2.19    2.8    1.25s
 10   -37.56368274956       -2.77       -2.34    2.8    900ms
 11   -37.56393451088       -3.60       -2.39    2.6    927ms
 12   -37.56428878524       -3.45       -2.55    1.6    791ms
 13   -37.56475086864       -3.34       -2.82    2.1    823ms
 14   -37.56488696230       -3.87       -2.99    2.5    1.19s
 15   -37

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.